# 01 – Explore Tree Geometry

This notebook is for **visualizing and understanding** the Christmas tree
polygon used in the Santa 2025 – Christmas Tree Packing Challenge.

Goals:
- Define the tree shape in local coordinates.
- Visualize the polygon.
- Compute and plot the **circumscribed circle** (bounding radius).
- Inspect rotated copies of the tree.

The origin `(0, 0)` is defined as the **center of the top of the trunk**,
which is consistent with the competition definition for `(x, y)` coordinates.

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
from pathlib import Path

%matplotlib inline


In [ ]:
# Tree polygon in local coordinates
# Origin (0, 0) = center of the top of the trunk.

TREE_TEMPLATE_VERTS = np.array([
    [0.0, 0.8],          # Tip of the tree
    [0.25 / 2, 0.5],     # Right top tier
    [0.25 / 4, 0.5],
    [0.4 / 2, 0.25],     # Right middle tier
    [0.4 / 4, 0.25],
    [0.7 / 2, 0.0],      # Right bottom tier
    [0.15 / 2, 0.0],     # Right trunk top
    [0.15 / 2, -0.2],    # Right trunk bottom
    [-0.15 / 2, -0.2],   # Left trunk bottom
    [-0.15 / 2, 0.0],    # Left trunk top
    [-0.7 / 2, 0.0],     # Left bottom tier
    [-0.4 / 4, 0.25],    # Left middle tier
    [-0.4 / 2, 0.25],
    [-0.25 / 4, 0.5],    # Left top tier
    [-0.25 / 2, 0.5],
], dtype=float)

TREE_RADIUS = float(np.linalg.norm(TREE_TEMPLATE_VERTS, axis=1).max())
TREE_RADIUS


In [ ]:
def make_tree_polygon_local():
    """Return the tree polygon in local coordinates as a shapely Polygon."""
    return Polygon(TREE_TEMPLATE_VERTS)

tree_poly = make_tree_polygon_local()
tree_poly


In [ ]:
def plot_tree_with_circle(poly, radius, title="Tree and bounding circle", figsize=(5, 7)):
    xs, ys = poly.exterior.xy
    fig, ax = plt.subplots(figsize=figsize)
    ax.fill(xs, ys, alpha=0.4)
    ax.plot(xs, ys, color='black', linewidth=1.0)

    circle = plt.Circle((0.0, 0.0), radius, fill=False, linestyle='--', linewidth=1.5)
    ax.add_patch(circle)

    pad = radius * 0.2
    ax.set_xlim(-radius - pad, radius + pad)
    ax.set_ylim(-radius - pad, radius + pad)
    ax.set_aspect('equal', adjustable='box')
    ax.axhline(0.0, color='grey', linewidth=0.5)
    ax.axvline(0.0, color='grey', linewidth=0.5)
    ax.set_title(title)
    plt.show()

plot_tree_with_circle(tree_poly, TREE_RADIUS)


In [ ]:
def rotate_tree(poly, angle_deg):
    """Rotate the tree polygon around the origin by angle_deg."""
    return rotate(poly, angle=angle_deg, origin=(0.0, 0.0), use_radians=False)

angles = [0, 45, 90, 135]
fig, axes = plt.subplots(2, 2, figsize=(8, 10))
for ax, ang in zip(axes.flatten(), angles):
    rp = rotate_tree(tree_poly, ang)
    xs, ys = rp.exterior.xy
    ax.fill(xs, ys, alpha=0.4)
    ax.plot(xs, ys, color='black', linewidth=1.0)
    circ = plt.Circle((0.0, 0.0), TREE_RADIUS, fill=False, linestyle='--', linewidth=1.0)
    ax.add_patch(circ)
    pad = TREE_RADIUS * 0.2
    ax.set_xlim(-TREE_RADIUS - pad, TREE_RADIUS + pad)
    ax.set_ylim(-TREE_RADIUS - pad, TREE_RADIUS + pad)
    ax.set_aspect('equal', adjustable='box')
    ax.set_title(f"Rotation {ang}°")
    ax.axis('off')
plt.tight_layout()
plt.show()


In [ ]:
def place_tree(x, y, angle_deg):
    """Return a rotated+translated tree polygon for given pose."""
    base = rotate_tree(tree_poly, angle_deg)
    return translate(base, xoff=x, yoff=y)

poses = [
    (0.0, 0.0, 0.0),
    (1.0, 0.5, 45.0),
    (-1.0, -0.2, 90.0),
]

fig, ax = plt.subplots(figsize=(6, 6))
for (x, y, ang) in poses:
    p = place_tree(x, y, ang)
    xs, ys = p.exterior.xy
    ax.fill(xs, ys, alpha=0.4)
    ax.plot(xs, ys, linewidth=0.8)
    ax.text(x, y, f"({x:.1f}, {y:.1f})", fontsize=8, ha='center')

ax.set_aspect('equal', adjustable='box')
ax.set_title("Example tree placements")
ax.grid(True, linestyle='--', linewidth=0.3)
plt.show()
